In [6]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import numpy as np
import pymongo

In [7]:
# By just looking, I know that 1991-2020 are the years of beard data that is available on their website (2021 doesn't have anything because the awards were cancelled)
length = 2021-1991
years = [i+1991 for i in range(length)]

In [8]:
# After manually reviewing the data fields returned on the beard website, and the order in which they occur
cols_simple = ["honoree name","blank","Award Category","name of work/effort","location of effort/who foot the bill","type of work","achievement status","year"]

In [9]:
#GOT THIS GREAT CODE SNIPPET FROM https://stackoverflow.com/questions/11553721/using-a-string-variable-as-a-variable-name
#IT'S VERY SIMPLE, BUT IT BLEW MY MIND. ANYWAYS I USE IT BELOW
    #my_data = {}
    #foo = "hello"
    #my_data[foo] = "goodbye"

In [10]:
listie_dicts = []
for year in years:
    
    #first loop iterates through the years of beard data available, it reads through the first page by calling 'page = 1' below
    #we then save all awards as a bs4 result set by targetting the attribute "data-award-template" in the "div" tag
    #in order to account for pagination, we target the second to last "li" tag with the 'page-item' class to determine the # of pages
    #needing to be scraped.
    page = 1
    
    #lets user know where parsing is at
    print(f"on {year}")
    url = f'https://www.jamesbeard.org/awards/search?keyword=&year={year}&page={page}'
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    awarding = soup.find_all(attrs={"data-award-template":'components.search-results.award.book.book'})
    pages = soup.find_all("li",class_='page-item')[-2].get_text()
    
    #we then parse through our bs4 result set and narrow all the way down to single data points on single awards
    #this parses only through the first page
    for i in awarding:
        
        #'p' tags are where all of the awards are stored            
        plop = i.find_all('p')
        
        #initialize the dictionary for the individual award information to be stored
        award = {}
        
        #obtain the data-award-template value that is only available in the 'div' tag
        uid = i.get('data-award-template')
        
        #establish data quality fields
        award['award_type'] = uid
        award['length'] = len(plop)
        award['pagenum'] = 1
        
        for count, j in enumerate(plop):
            
            key = cols_simple[count]
            award[key] = j.text.strip()
            
        listie_dicts.append(award)
   
    #This then takes the page length information gathered from the first scrape and scrapes and parses through the exact same way through all
    #pages for that year of award data.
    for page in range(2,int(pages)+1):
        
        #lets user know where parsing is at
        print(f"on {page} of {int(pages)}")
        
        url = f'https://www.jamesbeard.org/awards/search?keyword=&year={year}&page={page}'
        html = requests.get(url)
        soup = BeautifulSoup(html.text, 'html.parser')
        
        
        awarding = soup.find_all(attrs={"data-award-template":'components.search-results.award.book.book'})
        
        for i in awarding:
            
            #'p' tags are where all of the awards are stored
            plop = i.find_all('p')
            
            #initialize the dictionary for the individual award information to be stored
            award = {}
            
            #obtain the data-award-template value that is only available in the 'div' tag
            uid = i.get('data-award-template')
            
            #establish data quality fields
            award['award_type'] = uid
            award['length'] = len(plop)
            award['pagenum'] = page
            
            for count, j in enumerate(plop):
                
                #cols_simple are the attribute names I wanted data points stored as... they were originally made to be agnostic to award-type
                #but the current scraper grabs only one data-award-template (planning to implement scraper of all types with code blocks at end of ipynb)
                key = cols_simple[count]
                award[key] = j.text.strip()
                
            listie_dicts.append(award)
            
#after all of the years and pages have been scraped and parsed, it is saved into a dataframe for further cleaning
df = pd.DataFrame(listie_dicts)

on 1991
on 2 of 2
on 1992
on 2 of 2
on 1993
on 2 of 2
on 1994
on 2 of 3
on 3 of 3
on 1995
on 2 of 3
on 3 of 3
on 1996
on 2 of 3
on 3 of 3
on 1997
on 2 of 3
on 3 of 3
on 1998
on 2 of 3
on 3 of 3
on 1999
on 2 of 3
on 3 of 3
on 2000
on 2 of 3
on 3 of 3
on 2001
on 2 of 3
on 3 of 3
on 2002
on 2 of 3
on 3 of 3
on 2003
on 2 of 3
on 3 of 3
on 2004
on 2 of 3
on 3 of 3
on 2005
on 2 of 3
on 3 of 3
on 2006
on 2 of 4
on 3 of 4
on 4 of 4
on 2007
on 2 of 4
on 3 of 4
on 4 of 4
on 2008
on 2 of 7
on 3 of 7
on 4 of 7
on 5 of 7
on 6 of 7
on 7 of 7
on 2009
on 2 of 7
on 3 of 7
on 4 of 7
on 5 of 7
on 6 of 7
on 7 of 7
on 2010
on 2 of 7
on 3 of 7
on 4 of 7
on 5 of 7
on 6 of 7
on 7 of 7
on 2011
on 2 of 7
on 3 of 7
on 4 of 7
on 5 of 7
on 6 of 7
on 7 of 7
on 2012
on 2 of 7
on 3 of 7
on 4 of 7
on 5 of 7
on 6 of 7
on 7 of 7
on 2013
on 2 of 7
on 3 of 7
on 4 of 7
on 5 of 7
on 6 of 7
on 7 of 7
on 2014
on 2 of 7
on 3 of 7
on 4 of 7
on 5 of 7
on 6 of 7
on 7 of 7
on 2015
on 2 of 7
on 3 of 7
on 4 of 7
on 5 of 7
on 6 of 7


In [11]:
#save for backup
df.to_csv('CSV_Library/Beard_Book_scrape.csv')

In [12]:
#takes the awards with 7 fields and inserts nan's where the field is missing according to user review, this will put all fields in the correct columns for len = 7 awards

## THIS IS TO BE RUN ONE TIME, IF IT IS RUN MORE THAN ONCE, IT MUST THEN BE RUN UNTIL THE ARRAYS BEGIN WITH nan according to what is printed in insert

insert = df.loc[df['length'] == 7, 'location of effort/who foot the bill':'year'].values

for count, i in enumerate(insert):
    val = i[3]
    i[3] = i[2]
    i[2] = i[1]
    i[1] = i[0]
    i[0] = val    
df.loc[df['length'] ==7, df.columns[7:11]] = insert

insert

array([[nan, 'Book', 'Winner', '1991'],
       [nan, 'Book', 'Winner', '1992'],
       [nan, 'Book', 'Winner', '1992'],
       [nan, 'Book', 'Winner', '1993'],
       [nan, 'Book', 'Winner', '1993'],
       [nan, 'Book', 'Winner', '1994'],
       [nan, 'Book', 'Winner', '1995'],
       [nan, 'Book', 'Winner', '1996'],
       [nan, 'Book', 'Winner', '1997'],
       [nan, 'Book', 'Winner', '1998'],
       [nan, 'Book', 'Winner', '1999'],
       [nan, 'Book', 'Winner', '2000'],
       [nan, 'Book', 'Winner', '2001'],
       [nan, 'Book', 'Winner', '2002'],
       [nan, 'Book', 'Winner', '2003'],
       [nan, 'Book', 'Winner', '2004'],
       [nan, 'Book', 'Winner', '2005'],
       [nan, 'Book', 'Winner', '2006'],
       [nan, 'Book', 'Winner', '2007'],
       [nan, 'Book', 'Winner', '2008'],
       [nan, 'Book', 'Winner', '2009'],
       [nan, 'Book', 'Winner', '2010'],
       [nan, 'Book', 'Winner', '2011'],
       [nan, 'Book', 'Winner', '2012'],
       [nan, 'Book', 'Winner', '2013'],


In [13]:
# after all len 7 awards have been corrected, we will now change their length to 8
df.loc[df['length'] == 7, 'length'] = 8

In [14]:
#as data continues to be uploaded and new awards are given, there may be other length book fields (this is also in preparation for building out award scraping beyond just book awards)
# this will let user know whether or not there may be additional data transformations needed to the data frame.
if len(df['length'].value_counts()) <= 1:
    print("proceed")
else:
    print("addition transformations need to be made")

proceed


In [15]:
#for future implementation of wiki api integration
df['wiki_name'] = df['honoree name'].str.replace(' ', '_')
df['wiki_url'] = [f"https://en.wikipedia.org/wiki/{i}" for i in df['wiki_name'] ]
df.drop(labels=['wiki_name','blank'],axis=1,inplace=True)
df

,award_type,length,pagenum,honoree name,Award Category,name of work/effort,location of effort/who foot the bill,type of work,achievement status,year,wiki_url
0,components.search-results.award.book.book,8,1,The Editors of Better Homes & Gardens,Baking & Desserts,Old Fashioned Home Baking,(Meredith Corporation),Book,Nominee,1991,https://en.wikipedia.org/wiki/The_Editors_of_B...
1,components.search-results.award.book.book,8,1,The Editors of Sunset Magazine,Health and Diet,Light & Healthy Cook Book,(Sunset Publishing Corporation),Book,Nominee,1991,https://en.wikipedia.org/wiki/The_Editors_of_S...
2,components.search-results.award.book.book,8,1,The Moosewood Collective,"Fruits, Vegetables & Grains",Sundays at Moosewood Restaurant,(Fireside/Simon & Schuster),Book,Nominee,1991,https://en.wikipedia.org/wiki/The_Moosewood_Co...
3,components.search-results.award.book.book,8,1,Burton Anderson,Wine & Spirits,The Wine Atlas of Italy: A Traveler's Guide to...,(Simon & Schuster),Book,Winner,1991,https://en.wikipedia.org/wiki/Burton_Anderson
4,components.search-results.award.book.book,8,1,Jean Anderson,Techniques,Microways,(Doubleday),Book,Nominee,1991,https://en.wikipedia.org/wiki/Jean_Anderson
...,...,...,...,...,...,...,...,...,...,...,...
1426,components.search-results.award.book.book,8,2,Editors at America's Test Kitchen,Vegetable-Focused Cooking,Vegetables Illustrated,(America's Test Kitchen),Book,Nominee,2020,https://en.wikipedia.org/wiki/Editors_at_Ameri...
1427,components.search-results.award.book.book,8,4,Kaitlyn Goalen,American,Cook Like a Local,(Clarkson Potter),Book,Nominee,2020,https://en.wikipedia.org/wiki/Kaitlyn_Goalen
1428,components.search-results.award.book.book,8,4,Jancis Robinson,Cookbook Hall of Fame,,NaN,Book,Winner,2020,https://en.wikipedia.org/wiki/Jancis_Robinson
1429,components.search-results.award.book.book,8,4,Abra Berens,Vegetable-Focused Cooking,Ruffage,(Chronicle Books),Book,Nominee,2020,https://en.wikipedia.org/wiki/Abra_Berens


In [16]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://127.0.0.1:27017/'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.beard_db
collection = db.awards
#remove all stored information & upload via df (this should be updated with an index rule so when it is queried, only new records are entered into the database, and no old records are deleted)
collection.delete_many({})
collection.insert_many(df.to_dict('records'))

In [19]:
#check to see if insert many worked
listings = db.awards.find()
test_list =[]
for listing in listings:
    test_list.append(listing)
test_df = pd.DataFrame(test_list)

In [51]:
#This module takes all of our winning persons and randomly selects 100 unique authors for analysis. 
import random

winners_unique = test_df.loc[test_df['achievement status'] == "Winner",['honoree name']].value_counts()

winner_list = []
while len(winner_list) < 100:
    random_index = random.randint(0,len(winners_unique)-1)
    winner = winners_unique.index[random_index][0]
    if winner in winner_list:
        continue
    else:
        winner_list.append(winner)


pd.DataFrame(winner_list).to_csv("C:/Users/JoshWeidenaar/Documents/Bootcamp/1_Homework/Project-2/listof100.csv")

In [78]:
test_df["api_name"] = [i.split("/")[-1].replace("_","%20") for i in test_df['wiki_url']]

In [84]:
honoree_names = test_df['api_name'].value_counts().index.tolist()

'https://en.wikipedia.org/wiki/Christopher_Schlesinger'

In [85]:
winner_list
#df.to_csv("CSV_Library/beard_books_final.csv")

['Diana Miller',
 'Madhur Jaffrey',
 'Jacqueline Friedrich',
 'Alan Tardi',
 'Dorothy Kalins',
 'Julee Rosso',
 "Molly O'Neill",
 'Molly Stevens',
 'Catherine Donnelly',
 'Arrigo Cipriani',
 'Melissa Hamilton',
 'Bren Smith',
 'Shirley Corriher',
 'Deborah Madison',
 'Ellie Krieger',
 'Bert Greene',
 'Jack Czarnecki',
 'Rebecca Cate',
 'Sami Tamimi',
 'Vicky Bennison',
 'Steven Jenkins',
 'George Greenstein',
 'Christopher Schlesinger',
 'Julia Harding',
 'Steven Raichlen',
 'Oretta Zanini de Vita',
 'Barbara Kafka',
 'Stella Parks',
 'Waverly Root',
 'Larry Forgione',
 'Andre Domine',
 'Gil Marks',
 'Randall Grahm',
 'Hugh Johnson',
 'Amy Chaplin',
 'Maida Heatter',
 'Peter Johnson',
 'Michael Weiss',
 'Marlene Divina',
 'John Ash',
 'Nina Simonds',
 'Barbara Tropp',
 'Paul Greenberg',
 'Paul Lukacs',
 'Deborah Krasner',
 'J. Kenji Lopez-Alt',
 'John Doc Willoughby',
 'Joshua McFadden',
 'David Kaplan',
 'Richard Sax',
 'Marcella Hazan',
 'Beatrice Ojakangas',
 'Marlene Sorosky Gray',

In [113]:
# Wikiscrape Module


listie_dicts = []
for i in honoree_names:

    WikiMedia_URL = f"https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&ppprop=wikibase_item&titles={i}&formatversion=2&format=json"
    response = requests.get(WikiMedia_URL).json()
    id_for_both = response['query']['pages'][0]
    missing_key = 'missing'
    found_key = 'pageprops'
    honoree_dict = {}
    honoree_dict['name'] = i
    if found_key in id_for_both.keys():
        found_ID = id_for_both['pageprops']['wikibase_item']
        honoree_dict['id'] = found_ID
    elif missing_key in id_for_both.keys():
        missing_ID = id_for_both['missing']
        honoree_dict['id'] = missing_ID
    else: 
        print(f"somethings wrong with {i}")
    listie_dicts.append(honoree_dict)
            

somethings wrong with King%20Arthur%20Flour
somethings wrong with Ted%20Lee
somethings wrong with Matt%20Lee
somethings wrong with Andrea%20Immer%20Robinson
somethings wrong with Michele%20Evans
somethings wrong with Grace%20Young
somethings wrong with Pam%20Anderson
somethings wrong with Pierre%20Herme
somethings wrong with Peter%20Meehan
somethings wrong with Julia%20Harding
somethings wrong with Michael%20Twitty
somethings wrong with Rene%20Redzepi
somethings wrong with Michael%20Coe
somethings wrong with George%20Taber
somethings wrong with A.%20Zee
somethings wrong with Janet%20Podleski
somethings wrong with Zarela%20Martinez
somethings wrong with Raghavan%20Iyer
somethings wrong with Davia%20Nelson
somethings wrong with Lisa%20Weiss
somethings wrong with José%20Vouillamoz
somethings wrong with Jeni%20Britton%20Bauer
somethings wrong with Felipe%20Fernandez-Armesto
somethings wrong with Greta%20Podleski
somethings wrong with J.%20Kenji%20Lopez-Alt
somethings wrong with Bill%20Kim


In [129]:
df = pd.DataFrame(listie_dicts).loc[pd.DataFrame(listie_dicts)['id'] != True,]

In [144]:
df = df.loc[pd.notna(df['id']),]

In [185]:
Sex_ID = "P21"
gender_possibilities = { "Q6581072" : 'Female',
 "Q6581097" : 'Male',
 "Q1097630" : 'Intersex',
 'Q1052281' : 'Transgender_Female',
 'Q2449503' : 'Transgender_Male'}

In [216]:
input_possibilities = list(gender_possibilities.values())

nums = [input_possibilities.index(i) for i in input_possibilities]

text_forinput = "0 == Female\n\n 1 == Male\n\n 2 == Intersex\n\n 3 == Transgender_Female\n\n 4 == Transgender_Male"

df.reset_index(inplace=True)

In [243]:
author_gender_list = []
for idx, i in enumerate(df['id']):
    print(f"author: {df['name'][idx]} \n{(idx/len(df))*100}% of the way there")
    WikiData_URL = f"https://m.wikidata.org/wiki/Special:EntityData/{i}.json"
    response = requests.get(WikiData_URL).json()
    try:
        property_verification = response['entities'][i]['claims'][Sex_ID][0]['mainsnak']['property']
        gender_found = response['entities'][i]['claims'][Sex_ID][0]['mainsnak']['datavalue']['value']['id']

        if gender_found in gender_possibilities.keys():
            gender_found = gender_possibilities[gender_found]
        elif idx == 10:
            break
        else:
            gender_found = "NOTFOUND"
        author_gender_list.append(gender_found)
    except KeyError:
        print(f"{i} prompted a KeyError \n please enter gender assumed gender of {df['name'][idx]}\n\n\n{text_forinput} ")
        author_gender_list.append(input_possibilities[int(input())])

    

author: Steven%20Raichlen 
0.0% of the way there
author: James%20Peterson 
0.2597402597402597% of the way there
Q6141197 prompted a KeyError 
 please enter gender assumed gender of James%20Peterson


 0 == Female

 1 == Male

 2 == Intersex

 3 == Transgender_Female

 4 == Transgender_Male 


 1


author: Deborah%20Madison 
0.5194805194805194% of the way there
author: Naomi%20Duguid 
0.7792207792207793% of the way there
author: Dorie%20Greenspan 
1.0389610389610389% of the way there
author: Jacques%20Pépin 
1.2987012987012987% of the way there
author: Madhur%20Jaffrey 
1.5584415584415585% of the way there
author: Anne%20Willan 
1.8181818181818181% of the way there
author: Alice%20Medrich 
2.0779220779220777% of the way there
author: Jeffrey%20Alford 
2.3376623376623376% of the way there
author: Jean%20Anderson 
2.5974025974025974% of the way there
author: Rozanne%20Gold 
2.857142857142857% of the way there
author: Michael%20Ruhlman 
3.116883116883117% of the way there
author: Yotam%20Ottolenghi 
3.3766233766233764% of the way there
author: Mark%20Bittman 
3.6363636363636362% of the way there
author: Jancis%20Robinson 
3.896103896103896% of the way there
author: Claudia%20Roden 
4.1558441558441555% of the way there
author: Tim%20Turner 
4.415584415584416% of the way there
author:

 1


author: Andrew%20Dornenburg 
9.090909090909092% of the way there
author: Diana%20Kennedy 
9.35064935064935% of the way there
author: Oz%20Clarke 
9.61038961038961% of the way there
author: Ellie%20Krieger 
9.87012987012987% of the way there
author: Clive%20Coates 
10.129870129870131% of the way there
author: Alex%20Day 
10.38961038961039% of the way there
author: Rick%20Bayless 
10.649350649350648% of the way there
author: Sally%20Schneider 
10.909090909090908% of the way there
author: Marion%20Cunningham 
11.168831168831169% of the way there
Q6765162 prompted a KeyError 
 please enter gender assumed gender of Marion%20Cunningham


 0 == Female

 1 == Male

 2 == Intersex

 3 == Transgender_Female

 4 == Transgender_Male 


...

author: Constance%20Jones 
97.40259740259741% of the way there
author: Barbara%20Smith 
97.66233766233766% of the way there
author: Evan%20Fraser 
97.92207792207792% of the way there
Q26905834 prompted a KeyError 
 please enter gender assumed gender of Evan%20Fraser


 0 == Female

 1 == Male

 2 == Intersex

 3 == Transgender_Female

 4 == Transgender_Male 


 1


author: Jeff%20Cox 
98.18181818181819% of the way there
author: Adrian%20Miller 
98.44155844155844% of the way there
author: Nikki%20Silva 
98.7012987012987% of the way there
author: Art%20Smith 
98.96103896103897% of the way there
Q4797076 prompted a KeyError 
 please enter gender assumed gender of Art%20Smith


 0 == Female

 1 == Male

 2 == Intersex

 3 == Transgender_Female

 4 == Transgender_Male 


 1


author: Susan%20Branch 
99.22077922077922% of the way there
author: David%20Chang 
99.48051948051948% of the way there
author: Nancy%20Silverton 
99.74025974025975% of the way there


In [247]:
#change Jack Turner to MALE
df['gender'] = author_gender_list

<ipython-input-247-0413b7dc5651>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'] = author_gender_list


In [250]:
df['matchname'] = [i.replace('%20'," ") for i in df['name']]

<ipython-input-250-e43ad4d31a0e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['matchname'] = [i.replace('%20'," ") for i in df['name']]


In [251]:
df

,index,name,id,gender,matchname
0,0,Steven%20Raichlen,Q606656,Male,Steven Raichlen
1,1,James%20Peterson,Q6141197,Male,James Peterson
2,2,Deborah%20Madison,Q5248318,Female,Deborah Madison
3,3,Naomi%20Duguid,Q6964634,Female,Naomi Duguid
4,4,Dorie%20Greenspan,Q5297821,Female,Dorie Greenspan
...,...,...,...,...,...
380,967,Nikki%20Silva,Q7035262,Female,Nikki Silva
381,968,Art%20Smith,Q4797076,Male,Art Smith
382,969,Susan%20Branch,Q7647610,Female,Susan Branch
383,977,David%20Chang,Q3017652,Male,David Chang


In [253]:
df_final = pd.merge(test_df,df,how='left',left_on='honoree name',right_on='matchname')

In [254]:
len(df_final.loc[pd.notnull(df_final['gender']),])

693

In [266]:
df_final.drop(axis=1,labels=['api_name','name','matchname','index'],inplace=True)

In [269]:
df_final.to_csv('Beard_db_final.csv')

In [272]:
df_final.to_json('Beard_db_final.json', default_handler=str,orient="records")

In [201]:
# #this is to run through all award not just books

# Year_CodeBlock_awards = soup.find_all('div',class_="c-award-recipient")

# listie_dicts = []
# for i in Year_CodeBlock_awards:
#     plop = i.find_all('p')
#     award = {}
#     uid = i.get('data-award-template')
#     award['award_type'] = uid
#     award['length'] = len(plop)
#     for count, j in enumerate(plop):
#         key = cols_simple[count]
#         award[key] = j.text.strip()
#     listie_dicts.append(award)
# allAward_df = pd.DataFrame(listie_dicts)
# types = df['award_type'].unique().tolist()

# #this finds all award types and makes a dictonary of award types as keys and values as lists of tags officially a "bs4.element.ResultSet" 
# # IT IS NOT CONNECTED TO anything but the last soup object

# #ONE LINE DOWN IS HOW TYPES WAS ORIGINALLY GRABBED, I BELIEVE BS4 can probably do this more efficiently
# #types = df['award_type'].unique().tolist()

# dictie = {}
# for i in types:
#     key = i
#     dictie[key] = soup.find_all(attrs={"data-award-template":i})

# listie_dicts = []
# for i in dictie:
#     for j in dictie[i]:
#         plop = j.find_all('p')
#         award = {}
#         award['award_type'] = uid
#         award['length'] = len(plop)
#         for count, v in enumerate(plop):
#             key = cols_simple[count]
#             award[key] = v.text.strip()
#         listie_dicts.append(award)
    

In [ ]:
#iterates through single year awards and pulls out 1 single award
# 9 keys in dict, first is "tag" which identifies the div tags "data-award-template" value (this is in the first loop), 
#then it goes into the second loop and enters in the remaining 8 data points
#this is now the initial nested loop, this also checks for length issues, suggest currently, reviewing and then redoing according to "UID" variable rules.
##V1
# listie_dicts = []
# for i in Year_CodeBlock_awards:
#     listie_dicts.append(award)
#     plop = i.find_all('p')
#     award = {}
#     uid = i.get('data-award-template')
#     award['award_type'] = uid
#     for count, j in enumerate(plop):
#         key = cols_simple[count]
#         if len(plop) != 8:
#             award["lengthofissue"] = len(plop)
#             award[key] = j.text.strip()
# #         if len(plop) == 7 & count == 4:
            
# # #             print(f"THIS AWARD IS MISSING SOME INFORMATION\n\nTHE AWARD CODE BLOCK:\n\n{i}\n\n\n\n\n \nWHAT IS THE KEY THAT IS MISSING? \n\n\n\n\n THE AVAILABLE KEYS \n{cols_simple}") 
# # #             key = input()
# # #             print(f"\n\nTHE AWARD CODE BLOCK:\n\n{i}\n\n\n\n\n PLEASE ENTER THE VALUE YOU WOULD LIKE STORED FOR THE MISSING VARIABLE")
# #               award[key] = np.nan
# #               award["lengthofissue"] = len(plop)
# #         elif len(plop) == 6 & count == 4:
#         else:
#             award[key] = j.text.strip() 
            
# df = pd.DataFrame(listie_dicts)  

In [ ]:
##V2
# listie_dicts = []
# for i in Year_CodeBlock_awards:
#     plop = i.find_all('p')
#     award = {}
#     uid = i.get('data-award-template')
#     award['award_type'] = uid
#     award['length'] = len(plop)
#     for count, j in enumerate(plop):
#         key = cols_simple[count]
#         if (uid == "components.search-results.award.rnc.restaurant") & (count == 3):
#             award[key] = np.nan
# #         elif (uid == "components.search-results.award.rnc.humanitarian") & (count == 3 | 4):
# #             award[key] = np.nan
# #             award["length"] = len(plop)
#         else:
#             award[key] = j.text.strip()
#     listie_dicts.append(award)
# df2 = pd.DataFrame(listie_dicts)  

In [ ]:
##V3
# listie_dicts = []
# for i in Year_CodeBlock_awards:
#     plop = i.find_all('p')
#     award = {}
#     uid = i.get('data-award-template')
#     award['award_type'] = uid
#     length = len(plop)
#     award['length'] = len(plop)
    
#     if length == 8:
        
#         for count, j in enumerate(plop):
            
#             key = cols_simple[count]
#             award[key] = j.text.strip()
            
#     elif uid == "components.search-results.award.rnc.restaurant":
        
#         for count, j in enumerate(plop):
            
#             key = cols_simple[count]
#             alphastep = cols_simple[count + 1]
            
#             if (count == 3):
                
#                 award[key] = np.nan
#                 award[alphastep] = j.text.strip()
                
#             elif (count > 3):
                
#                 award[alphastep] = j.text.strip()
                
#             else:
                
#                 award[key] = j.text.strip()
#     elif uid == "components.search-results.award.rnc.whos-who":
        
#         for i in range(8):
            
#             if i < len(plop):
#                 key = cols_simple[i]
#                 alphastep = cols_simple[i + 1]
#                 omegastep = cols_simple[i +2]
#                 negastep = cols_simple[i-1]
#                 if key == 'name of work/effort':
#                     award[key] = np.nan
#                     award[alphastep] = plop[i].text.strip()
#                     print(plop[i])
#                     print(alphastep)
#                 elif key == 'type of work':
#                     award[key] = np.nan
#                     award[alphastep] = plop[i - 1].text.strip()
#                     award[omegastep] = plop[i].text.strip()
#                     print(plop[i-1])
#                 else:
#                     award[key] = plop[i].text.strip()
                #award[key] = plop[i].text.strip()
#             key = cols_simple[count]
#             alphastep = cols_simple[count + 1]
#             omegastep = cols_simple[count + 2]
#             if (count == 3):
#                 award[key] = np.nan
#                 award[alphastep] = j.text.strip()
#                 award[omegastep] = 'ohno'

#             elif (count == 5):
                
#                 award[alphastep] = j.text.strip()

#             else:
#                 award[key] = j.text.strip()
                
#     listie_dicts.append(award)
# ddd = pd.DataFrame(listie_dicts)
# ddd.loc[ddd['award_type'] == "components.search-results.award.rnc.whos-who",]